In [2]:
import os
import pandas as pd
import imageio
from IPython.display import display, Video, clear_output
from ipywidgets import Button, HBox, Output, Layout
import ipywidgets as widgets
from tifffile import imread, imwrite
import cv2
from IPython.core.display import Image
import numpy as np
import time

# Load DataFrame or Create if doesn't exist
csv_path = '/home/spsalmon/pharynx_test/scripts/pumping_manual_old_experiment.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
else:
    df = pd.DataFrame(columns=["ImagePath", "ManualPumpingCount"])

# Function to convert tiff to mp4 and display
def display_video(tiff_path):
    time.sleep(2)
    output_path = 'temp_video.mp4'
    video = imread(tiff_path)
    for frame in video:
        with video_player:
            clear_output(wait=True)
            frame = frame/np.max(frame)
            frame = (frame*255).astype(np.uint8)

            # Define the new size
            new_width = frame.shape[1] * 5  # Double the width
            new_height = frame.shape[0] * 5  # Double the height

            # Resize using bilinear interpolation
            resized_img_bilinear = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

            _,ret = cv2.imencode('.jpg', resized_img_bilinear) 
            i = Image(data=ret)
            display(i)
            time.sleep(0.1)
        


list_videos = df['ImagePath'].dropna().tolist()
# Widgets

video_player = Output(layout=Layout(height="600px", max_width="400px"))
video_dropdown = widgets.Dropdown(options=list_videos, description='Select Video:')
play_button = widgets.Button(description="Play Video")
event_input = widgets.IntText(description='Pump Events:')
save_button = widgets.Button(description="Save")
ignore_button = widgets.Button(description="Ignore")

# Actions
def play_button_clicked(b):
    if video_dropdown.value:
        display_video(video_dropdown.value)

def save_button_clicked(b):
    index = df[df['ImagePath'] == video_dropdown.value].index[0]
    df.at[index, 'ManualPumpingCount'] = event_input.value
    df.to_csv(csv_path, index=False)
    print(f"Saved {event_input.value} pumps for {video_dropdown.value}")

def ignore_button_clicked(b):
    index = df[df['ImagePath'] == video_dropdown.value].index[0]
    df.at[index, 'ManualPumpingCount'] = -1
    df.to_csv(csv_path, index=False)
    print(f"Ignored {video_dropdown.value}")

play_button.on_click(play_button_clicked)
save_button.on_click(save_button_clicked)
ignore_button.on_click(ignore_button_clicked)

# Display
display(video_player)
display(video_dropdown, play_button, event_input, save_button, ignore_button)


Output(layout=Layout(height='600px', max_width='400px'))

Dropdown(description='Select Video:', options=('/mnt/external.data/TowbinLab/kstojanovski/20220401_Ti2_20x_160…

Button(description='Play Video', style=ButtonStyle())

IntText(value=0, description='Pump Events:')

Button(description='Save', style=ButtonStyle())

Button(description='Ignore', style=ButtonStyle())

Saved 20 pumps for /mnt/external.data/TowbinLab/kstojanovski/20220401_Ti2_20x_160-182-190_pumping_25C_20220401_173300_429/analysis/str_pharynx_videos/Time00117_Point0021_Channel470 nm_Seq8079.tiff


Saved 14 pumps for /mnt/external.data/TowbinLab/kstojanovski/20220401_Ti2_20x_160-182-190_pumping_25C_20220401_173300_429/analysis/str_pharynx_videos/Time00127_Point0025_Channel470 nm_Seq8772.tiff


Ignored /mnt/external.data/TowbinLab/kstojanovski/20220401_Ti2_20x_160-182-190_pumping_25C_20220401_173300_429/analysis/str_pharynx_videos/Time00127_Point0037_Channel470 nm_Seq8786.tiff


Saved 24 pumps for /mnt/external.data/TowbinLab/kstojanovski/20220401_Ti2_20x_160-182-190_pumping_25C_20220401_173300_429/analysis/str_pharynx_videos/Time00134_Point0020_Channel470 nm_Seq9248.tiff
